# Sysbench - CPU Workloads: Firecracker microVM vs Bare-Metal

In [1]:
import re
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [7]:
df_mvm = pd.read_csv('sysbench_microVM.csv')
df_mvm['Env'] = 'microVM'

df_bm = pd.read_csv('sysbench_baremetal.csv')
df_bm['Env'] = 'Bare Metal'

dff = pd.concat([df_bm, df_mvm], ignore_index=True)
dff.drop('Unnamed: 0', axis=1, inplace=True)
dff.head()

,Number of vCPUs,Number of Threads,Max Prime,Events per second,Total Time,Total Number of Events,Min Latency (ms),Avg Latency (ms),Max Latency (ms),95th Percentile Latency (ms),Total Execution Time across Threads (s),Avg Number of Events per Thread,Avg Execution Time per Thread (s),Env
0,32,1,10000,1331.60,30.0006,39954.0,0.73,0.75,1.72,0.77,29.97783,39954.0,29.9778,Bare Metal
1,32,1,50000,140.88,30.0004,4227.0,7.00,7.10,13.80,7.17,29.99201,4227.0,29.9920,Bare Metal
2,32,1,100000,53.27,30.0159,1599.0,18.60,18.77,26.94,18.95,30.00966,1599.0,30.0097,Bare Metal
3,32,1,500000,5.49,30.0628,165.0,181.12,182.18,194.72,186.54,30.06030,165.0,30.0603,Bare Metal
4,32,1,1000000,2.05,30.2638,62.0,485.56,488.11,507.48,493.24,30.26264,62.0,30.2626,Bare Metal


## Visualize: Average Latency

In [35]:
dff['Environment Config'] = dff['Env'] + ' - ' + dff['Number of vCPUs'].astype(str) + 'vCPUs'
dff_1mil = dff.loc[dff['Max Prime'] == 1000000]

fig = px.line(dff_1mil, x='Number of Threads', y='Avg Latency (ms)', color='Environment Config', markers=True)
fig.add_vline(x=2, line_width=0.5, line_dash="dot", line_color="red")
fig.add_vline(x=4, line_width=0.5, line_dash="dot", line_color="red")
fig.add_vline(x=15, line_width=0.5, line_dash="dot", line_color="green")
fig.add_vline(x=16, line_width=0.5, line_dash="dot", line_color="blue")
fig.add_vline(x=30, line_width=0.5, line_dash="dot", line_color="green")
fig.add_vline(x=32, line_width=0.5, line_dash="dot", line_color="blue")
fig['layout'].update(title='Average Latency for an Event by a Thread (Max Prime = 1mil)')
fig.show()

In [36]:
dff_100k = dff.loc[dff['Max Prime'] == 100000]

fig = px.line(dff_100k, x='Number of Threads', y='Avg Latency (ms)', color='Environment Config', markers=True)
fig.add_vline(x=2, line_width=0.5, line_dash="dot", line_color="red")
fig.add_vline(x=4, line_width=0.5, line_dash="dot", line_color="red")
fig.add_vline(x=15, line_width=0.5, line_dash="dot", line_color="green")
fig.add_vline(x=16, line_width=0.5, line_dash="dot", line_color="blue")
fig.add_vline(x=30, line_width=0.5, line_dash="dot", line_color="green")
fig.add_vline(x=32, line_width=0.5, line_dash="dot", line_color="blue")
fig['layout'].update(title='Average Latency for an Event by a Thread (Max Prime = 100k)')
fig.show()

## Visualize: Total Number of Events

In [37]:
fig = px.line(dff_1mil, x='Number of Threads', y='Total Number of Events', color='Environment Config', markers=True)
fig.add_vline(x=2, line_width=0.5, line_dash="dot", line_color="red")
fig.add_vline(x=4, line_width=0.5, line_dash="dot", line_color="red")
fig.add_vline(x=15, line_width=0.5, line_dash="dot", line_color="green")
fig.add_vline(x=16, line_width=0.5, line_dash="dot", line_color="blue")
fig.add_vline(x=30, line_width=0.5, line_dash="dot", line_color="green")
fig.add_vline(x=32, line_width=0.5, line_dash="dot", line_color="blue")
fig['layout'].update(title='Total Number of Events Computed (Max Prime = 1mil)')
fig.show()

In [38]:
fig = px.line(dff_100k, x='Number of Threads', y='Total Number of Events', color='Environment Config', markers=True)
fig.add_vline(x=2, line_width=0.5, line_dash="dot", line_color="red")
fig.add_vline(x=4, line_width=0.5, line_dash="dot", line_color="red")
fig.add_vline(x=15, line_width=0.5, line_dash="dot", line_color="green")
fig.add_vline(x=16, line_width=0.5, line_dash="dot", line_color="blue")
fig.add_vline(x=30, line_width=0.5, line_dash="dot", line_color="green")
fig.add_vline(x=32, line_width=0.5, line_dash="dot", line_color="blue")
fig['layout'].update(title='Total Number of Events Computed (Max Prime = 100k)')
fig.show()

## Visualize: CPU Speed (Events per second)

In [39]:
fig = px.line(dff_1mil, x='Number of Threads', y='Events per second', color='Environment Config', markers=True)
fig.add_vline(x=2, line_width=0.5, line_dash="dot", line_color="red")
fig.add_vline(x=4, line_width=0.5, line_dash="dot", line_color="red")
fig.add_vline(x=15, line_width=0.5, line_dash="dot", line_color="green")
fig.add_vline(x=16, line_width=0.5, line_dash="dot", line_color="blue")
fig.add_vline(x=30, line_width=0.5, line_dash="dot", line_color="green")
fig.add_vline(x=32, line_width=0.5, line_dash="dot", line_color="blue")
fig['layout'].update(title='CPU Speed (Eps) (Max Prime = 1mil)')
fig.show()

In [34]:
fig = px.line(dff_100k, x='Number of Threads', y='Events per second', color='Environment Config', markers=True)
fig.add_vline(x=2, line_width=1, line_dash="dot", line_color="red")
fig.add_vline(x=4, line_width=1, line_dash="dot", line_color="red")
fig.add_vline(x=15, line_width=1, line_dash="dot", line_color="green")
fig.add_vline(x=16, line_width=1, line_dash="dot", line_color="blue")
fig.add_vline(x=30, line_width=1, line_dash="dot", line_color="green")
fig.add_vline(x=32, line_width=1, line_dash="dot", line_color="blue")
fig['layout'].update(title='CPU Speed (Eps) (Max Prime = 100k)')
fig.show()